In [25]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

### **Modelo Matemático**

#### **Conjuntos:**
- \( C \): Conjunto de ciudades destino \( \{Cali, Barranquilla, Pasto, Tunja, Chía, Manizales\} \)
- \( O \): Conjunto de ciudades origen \( \{Bogotá, Medellín\} \)

#### **Parámetros:**
- $ d_{i} $: Demanda de la ciudad \( i \), donde $ i \in C $
- $ s_{j} $: Oferta de la ciudad \( j \), donde $ j \in O $
- $ c_{ij} $: Costo de transporte por tonelada entre la ciudad de origen \( j \) y la ciudad de destino \( i \)




In [26]:
# Conjuntos
C = ['Cali', 'Barranquilla', 'Pasto', 'Tunja', 'Chía', 'Manizales']
O = ['Bogotá', 'Medellín']

In [27]:
# Parámetros
demanda = {'Cali': 125, 'Barranquilla': 175, 'Pasto': 225, 'Tunja': 250, 'Chía': 225, 'Manizales': 200}
oferta = {'Bogotá': 550, 'Medellín': 700}

costos_transporte = {
    ('Cali', 'Bogotá'): 999, ('Cali', 'Medellín'): 2.5,
    ('Barranquilla', 'Bogotá'): 2.5, ('Barranquilla', 'Medellín'): 999,
    ('Pasto', 'Bogotá'): 1.6, ('Pasto', 'Medellín'): 2.0,
    ('Tunja', 'Bogotá'): 1.4, ('Tunja', 'Medellín'): 1.0,
    ('Chía', 'Bogotá'): 0.8, ('Chía', 'Medellín'): 1.0,
    ('Manizales', 'Bogotá'): 1.4, ('Manizales', 'Medellín'): 0.8
}

### **Variables de Decisión:**
Sea:
- $ x_{ij} $ la cantidad de toneladas de productos transportadas desde la ciudad \( j \) (origen) a la ciudad \( i \) (destino), donde $ i \in C $ y $ j \in O $.

### **Función Objetivo:**
El objetivo es minimizar el costo total de transporte:
$$
\text{Minimizar:} \quad \sum_{i \in C} \sum_{j \in O} c_{ij} \cdot x_{ij}
$$
Donde:
- $ c_{ij} $ es el costo de transportar una tonelada desde la ciudad \( j \) a la ciudad \( i \),
-  C es el conjunto de ciudades de destino,
- O es el conjunto de ciudades de origen.


In [28]:
# Crear el modelo
model = ConcreteModel()

# Variables de decisión
model.x = Var(C, O, domain=NonNegativeReals)

In [29]:
# Función objetivo: Minimizar el costo total de transporte
def costo_total(model):
    return sum(costos_transporte[(i, j)] * model.x[i, j] for i in C for j in O)

model.objetivo = Objective(rule=costo_total, sense=minimize)

### **Restricciones:**

1. **Restricción de Demanda:**
   Cada ciudad de destino debe recibir al menos la cantidad de productos requerida:
   $$
   \sum_{j \in O} x_{ij} \geq d_{i} \quad \forall i \in C
   $$
   Donde \( d_{i} \) es la demanda de la ciudad \( i \).

2. **Restricción de Oferta:**
   Cada ciudad de origen no puede enviar más productos de los que tiene disponibles:
   $$
   \sum_{i \in C} x_{ij} \leq s_{j} \quad \forall j \in O
   $$
   Donde $ s_{j} $ es la oferta de la ciudad \( j \).


In [30]:
# Restricción de demanda: cada ciudad debe recibir al menos su demanda
def demanda_satisfecha(model, i):
    return sum(model.x[i, j] for j in O) >= demanda[i]

model.restriccion_demanda = Constraint(C, rule=demanda_satisfecha)

In [31]:
# Restricción de oferta: cada ciudad de origen no puede enviar más productos de los que tiene disponibles
def oferta_disponible(model, j):
    return sum(model.x[i, j] for i in C) <= oferta[j]

model.restriccion_oferta = Constraint(O, rule=oferta_disponible)

In [32]:
# Resolver el modelo
solver = SolverFactory('glpk')
resultado = solver.solve(model)

# Mostrar los resultados de las variables de decisión
model.display()

Model unknown

  Variables:
    x : Size=12, Index={Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}*{Bogotá, Medellín}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Barranquilla', 'Bogotá') :     0 : 175.0 :  None : False : False : NonNegativeReals
        ('Barranquilla', 'Medellín') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Cali', 'Bogotá') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Cali', 'Medellín') :     0 : 125.0 :  None : False : False : NonNegativeReals
                  ('Chía', 'Bogotá') :     0 : 150.0 :  None : False : False : NonNegativeReals
                ('Chía', 'Medellín') :     0 :  75.0 :  None : False : False : NonNegativeReals
             ('Manizales', 'Bogotá') :     0 :   0.0 :  None : False : False : NonNegativeReals
           ('Manizales', 'Medellín') :     0 : 200.0 :  None : False : False : NonNegativeReals
                 ('Pasto

In [33]:
# Cargar duales
model.dual = Suffix(direction=Suffix.IMPORT)

# Resolver el modelo con duales
resultado = solver.solve(model)

In [34]:
#Mostrar duales de las restricciones
print("\nAnálisis de Sensibilidad:")
for i in C:
    print(f"Dual de la restricción de demanda en {i}: {model.dual[model.restriccion_demanda[i]]}")

for j in O:
    print(f"Dual de la restricción de oferta en {j}: {model.dual[model.restriccion_oferta[j]]}")



Análisis de Sensibilidad:
Dual de la restricción de demanda en Cali: 2.5
Dual de la restricción de demanda en Barranquilla: 2.7
Dual de la restricción de demanda en Pasto: 1.8
Dual de la restricción de demanda en Tunja: 1.0
Dual de la restricción de demanda en Chía: 1.0
Dual de la restricción de demanda en Manizales: 0.8
Dual de la restricción de oferta en Bogotá: -0.2
Dual de la restricción de oferta en Medellín: 0.0


In [35]:
# Mover 50 toneladas de oferta de Medellín a Bogotá y repetir el análisis de sensibilidad
print("\nMover 50 toneladas de oferta de Medellín a Bogotá\n")

oferta['Bogotá'] += 50
oferta['Medellín'] -= 50

# Resolver nuevamente con la nueva distribución de oferta
resultado = solver.solve(model)
model.display()



Mover 50 toneladas de oferta de Medellín a Bogotá

Model unknown

  Variables:
    x : Size=12, Index={Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}*{Bogotá, Medellín}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Barranquilla', 'Bogotá') :     0 : 175.0 :  None : False : False : NonNegativeReals
        ('Barranquilla', 'Medellín') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Cali', 'Bogotá') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Cali', 'Medellín') :     0 : 125.0 :  None : False : False : NonNegativeReals
                  ('Chía', 'Bogotá') :     0 : 150.0 :  None : False : False : NonNegativeReals
                ('Chía', 'Medellín') :     0 :  75.0 :  None : False : False : NonNegativeReals
             ('Manizales', 'Bogotá') :     0 :   0.0 :  None : False : False : NonNegativeReals
           ('Manizales', 'Medellín') :     0 : 200.0 :  None : False

In [36]:
# Mostrar duales de las nuevas restricciones
print("\nAnálisis de Sensibilidad después del cambio:")
for i in C:
    print(f"Dual de la restricción de demanda en {i}: {model.dual[model.restriccion_demanda[i]]}")

for j in O:
    print(f"Dual de la restricción de oferta en {j}: {model.dual[model.restriccion_oferta[j]]}")



Análisis de Sensibilidad después del cambio:
Dual de la restricción de demanda en Cali: 2.5
Dual de la restricción de demanda en Barranquilla: 2.7
Dual de la restricción de demanda en Pasto: 1.8
Dual de la restricción de demanda en Tunja: 1.0
Dual de la restricción de demanda en Chía: 1.0
Dual de la restricción de demanda en Manizales: 0.8
Dual de la restricción de oferta en Bogotá: -0.2
Dual de la restricción de oferta en Medellín: 0.0


# Análisis del Problema 1 - Optimización de Redes de Transporte

## 1. Resultados del Modelo Inicial

Los resultados muestran la cantidad de productos transportados desde cada ciudad de origen (Bogotá o Medellín) hacia las ciudades de destino:

- **De Bogotá**:
  - Chía: 150 toneladas.
  - Pasto: 225 toneladas.

- **De Medellín**:
  - Barranquilla: 175 toneladas.
  - Cali: 125 toneladas.
  - Chía: 75 toneladas.
  - Manizales: 200 toneladas.
  - Tunja: 250 toneladas.

El **costo total** del transporte en este caso es **1715 unidades monetarias**.

## 2. Análisis de los Duales (Multiplicadores de Lagrange)

Los valores duales indican cómo cambiaría el costo total si la oferta o la demanda de una ciudad cambia en una unidad. A continuación se muestran los duales obtenidos:

### Restricciones de demanda (ciudades de destino):
- **Cali**: 2.5
- **Barranquilla**: 2.7
- **Pasto**: 1.8
- **Tunja**: 1.0
- **Chía**: 1.0
- **Manizales**: 0.8

**Interpretación**:
- El valor dual de **Cali** es 2.5, lo que indica que aumentar su demanda en una tonelada incrementaría el costo total en 2.5 unidades monetarias.
- **Barranquilla** tiene el dual más alto (2.7), lo que sugiere que un aumento en su demanda tendría un impacto significativo en el costo.
- **Pasto** tiene un valor dual de 1.8, indicando un aumento moderado en el costo si se incrementa la demanda.
- **Tunja** y **Chía** tienen un dual de 1.0, indicando un impacto menor pero relevante.
- **Manizales**, con un dual de 0.8, tendría el menor impacto al aumentar su demanda.

### Restricciones de oferta (ciudades de origen):
- **Bogotá**: -0.2
- **Medellín**: 0.0

**Interpretación**:
- Un valor dual negativo en **Bogotá** (-0.2) indica que aumentar la oferta en Bogotá reduciría el costo total. Aumentar la oferta en una tonelada reduciría el costo en 0.2 unidades monetarias.
- El valor dual de **Medellín** es 0.0, lo que sugiere que aumentar su oferta no afectaría el costo total, ya que su oferta está bien utilizada.

## 3. Análisis de Sensibilidad después de Mover 50 Toneladas

Se realizó un análisis de sensibilidad moviendo 50 toneladas de oferta de Medellín a Bogotá. Los resultados del nuevo modelo fueron:

- El costo total **se mantiene en 1715 unidades monetarias**, lo que indica que el cambio no afectó significativamente los costos.

Los duales para las ciudades de destino y origen permanecen iguales, lo que sugiere que la redistribución no afectó las restricciones activas del modelo:

### Duales después del cambio:
- **Cali**: 2.5
- **Barranquilla**: 2.7
- **Pasto**: 1.8
- **Tunja**: 1.0
- **Chía**: 1.0
- **Manizales**: 0.8
- **Bogotá**: -0.2
- **Medellín**: 0.0

## Conclusión

El análisis con los duales nos muestra que:
- Aumentar la demanda en **Barranquilla** y **Cali** incrementaría significativamente el costo, debido a sus altos valores duales.
- Aumentar la oferta en **Bogotá** reduciría ligeramente el costo total, pero el impacto es mínimo.
- Redistribuir la oferta entre Medellín y Bogotá no afecta el costo total, ya que la oferta de Medellín está bien aprovechada y Bogotá tiene un impacto leve en el costo.

Esto sugiere que no es necesario realizar grandes cambios en la distribución de la oferta para mejorar el costo total, y que las ciudades con mayores demandas, como Barranquilla y Cali, deberían ser las prioridades.
